<img src="Dau.png">

#### MASEF, University Paris-Dauphine 2021:   Bryan Delamour 

## Turbo Charging Monte Carlo

#### Ref: Turbocharging Monte Carlo pricing for the rough Bergomi model
Ryan McCrickerd, Mikko S. Pakkanen

In [1]:
import numpy as np
from cmath import * 
import pickle
from time import time 
import scipy.stats as sps
from ipynb.fs.full.HybridScheme import *

## Parameters

$
\begin{array}{l}
\text { Training set of size } 34.000 \text { and testing set of size } 6.000\\
\text { Rough Bergomi sample: }\left(\xi_{0}, \nu, \rho, H\right) \in \mathcal{U}[0.01,0.16] \times \mathcal{U}[0.5,4.0] \times \mathcal{U}[-0.95,-0.1] \times \mathcal{U}[0.025,0.5]\\
\text { Strikes: }\{0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5\}\\
\text { Maturities: }\{0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0\}
\end{array}
$

In [ ]:
strikes = np.arange(0.5,1.6,0.1)
maturities = np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2])
S0=1

In [ ]:
## Loading parameters 

model_parameters_training_File = "model_parameters_training_data.pkl"

with open(model_parameters_training_File, 'rb') as file:  
    model_parameters = pickle.load(file)

XI = model_parameters[:,0]
NU = model_parameters[:,1]
RHO = model_parameters[:,2]
H = model_parameters[:,3]

### Option price estimator

We consider the orthogonal separation of the rBergomi price process $S_{t}$ into $S_{t}^{1}$ and $S_{t}^{2}$, where


$$
S_{t}^{1}:=\mathcal{E}\left(\rho \int_{0}^{1} \sqrt{V_{u}} \mathrm{~d} W_{u}^{1}\right)_{t}, \quad S_{t}^{2}:=\mathcal{E}\left(\sqrt{1-\rho^{2}} \int_{0}^{1} \sqrt{V_{u}} \mathrm{~d} W_{u}^{2}\right)_{t}
$$

### Mixed estimator

#### Control variate

This separation facilitates our Mixed estimator, which we define using
$$
X=B S\left(\left(1-\rho^{2}\right) \int_{0}^{t} V_{u} \mathrm{~d} u ; S_{t}^{1}, k\right), \quad Y=B S\left(\rho^{2}\left(\hat{Q}_{n}-\int_{0}^{t} V_{u} \mathrm{~d} u\right) ; S_{t}^{1}, k\right)
$$

to consider price estimators $\hat{P}_{n}(k, t)$ of the following form under the rBergomi model
$$
\hat{P}_{n}(k, t):=\frac{1}{n} \sum_{i=1}^{n}\left(X_{i}+\hat{\alpha}_{n} Y_{i}\right)-\hat{\alpha}_{n} \mathbb{E}[Y], \quad \hat{\sigma}_{B S}^{n}(k, t)^{2} t=B S^{-1}\left(\hat{P}_{n}(k, t) ; 1, k\right)
$$

We compute $\hat{\alpha}_{n}$ and $\hat{Q}_{n}$ post-simulation from sampled $X_{i}, Y_{i}$ and $\left(\int_{0}^{t} V_{u} \mathrm{~d} u\right)$, using


$$
\hat{\alpha}_{n}:=-\frac{\sum_{i=1}^{n}\left(X_{i}-\bar{X}_{n}\right)\left(Y_{i}-\bar{Y}_{n}\right)}{\sum_{i=1}^{n}\left(Y_{i}-\bar{Y}_{n}\right)^{2}}, \quad \hat{Q}_{n}:=\sup \left\{\left(\int_{0}^{t} V_{u} \mathrm{~d} u\right)_{i}: i=1, \ldots, n\right\}
$$

#### Antithetic sampling

We draw a path of $W^{1}$ over the interval $[0, t]$, and appeal to the symmetry in distribution of $S_{t}^{1, \pm}$, defined by

\begin{aligned}
S_{t}^{1, \pm} &=\mathcal{E}\left\{\pm \rho \int_{0}^{t} \sqrt{V_{u}^{\pm}} \mathrm{d} W_{u}^{1}\right\}, \quad V_{t}^{\pm}=\xi_{0}(t) \exp \left(-\frac{\eta^{2}}{2} t^{2 \alpha+1}\right)\left(V_{t}^{\circ}\right)^{\pm 1} \\
V_{t}^{\circ} &=\exp \left(\eta W_{t}^{\alpha}\right)
\end{aligned}


In [ ]:
def BS_price(S,K,V):
    d1 = ( np.log(S)-np.log(K) + 0.5*V )*(1/np.sqrt(V))
    d2 = d1 - np.sqrt(V)
    p = S*sps.norm.cdf(d1) - K*sps.norm.cdf(d2)
    return(p)

For each sample of rough Bergomi parameters (40 000) , each strike and maturity in the grid (11x8), we will generate a mixed estimator of the call price using $M = 60 000$ samples.

In [ ]:
t1=time()

nb_samples = 4*10**4
N_disc = 100
T_ind = N_disc*maturities
T_ind=T_ind.astype(int)
M = 6*10**4

BS_prices = np.zeros( ( nb_samples, len(strikes), len(maturities) ) )


for i in range(nb_samples):
    m = M//2
    rB = rBergomi_h(paths = m , N = N_disc, T = 2, H = H[i])
    
    #Anthitetic sampling 
    dW_ = rB.dW()
    dW = np.r_[dW_, -dW_]
    
    W_h = rB.W_h(dW)
    V = rB.V(XI[i], NU[i], W_h)
    S1 = rB.S1(S0, RHO[i], dW, V)

    QV = np.cumsum(V, axis = 1)* (1/N_disc)
    QVT = QV[:, T_ind]

    Q = np.max(QVT, axis=0) + 1e-9

    ST = S1[:, T_ind]
    VT = V[:, T_ind]

    S1T = S1[:,T_ind]


    mat = np.zeros( ( len(strikes), M , len(maturities ) ) )
    c = np.zeros(len(maturities))
    for k in range(len(strikes)): #Mixed Estimator 
        X = BS_price(S1T, strikes[k], (1 - RHO[i]**2) * QVT)
        Y = BS_price(S1T, strikes[k], RHO[i]**2 * (Q - QVT))
        eY = BS_price(1., strikes[k], RHO[i]**2 * Q)

        # Asymptotically optimal weights
        for t in range(len(maturities)):      
            cov_mat = np.cov(X[:,t], Y[:,t])
            c[t] = - cov_mat[0,1] / cov_mat[1,1]

        mat[k,:,:] = X + c * (Y - eY) # Control variate
    
    P = (mat[:, :m, :] + mat[:, m:, :])*0.5
    BS_prices[i,:,:] = np.mean(P,axis=1)

    if (i%10==0):
        print(i)
        
    if (i%1000 == 0):
        BS_prices_training_File = "BS_prices_training_data.pkl"  

        with open(BS_prices_training_File, 'wb') as file:  
            pickle.dump(BS_prices, file)
        
       
t2=time()
print("Time (h)", (t2-t1)/3600)
    
# 119h 

## Saving Black Scholes prices

In [ ]:
# Saving prices 

BS_prices_training_File = "BS_prices_training_data.pkl"  

with open(BS_prices_training_File, 'wb') as file:
    pickle.dump(BS_prices, file)